# Обучение YOLO

Монтирование гугл диска

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Установка YOLO

In [ ]:
# Install yolo
!pip3 install ultralytics

Копирование датасета с гугл диска, путь замените на свой

In [ ]:
!mkdir dataset
!cp /content/drive/MyDrive/Fruits.v3i.yolov11.zip /content/dataset
!unzip /content/dataset/Fruits.v3i.yolov11.zip -d /content/dataset > /dev/null

Исправление путей в конфигурации датасета

In [ ]:
# Fix path
import yaml

train_path = '/content/dataset/train'
val_path = '/content/dataset/valid'
test_path = '/content/dataset/test'

yaml_path = '/content/dataset/data.yaml'

with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

data['train'] = train_path
data['val'] = val_path
data['test'] = test_path

with open(yaml_path, 'w') as f:
    yaml.safe_dump(data, f)


Стандартная процедура обучения. Можете поменять количество эпох, размер изображений и период сохранения промежуточных весов модели

In [ ]:
# Train

# Tune epochs, imgsz, save_T
from ultralytics import YOLO

model = YOLO("yolo11n.pt")
results = model.train(data="/content/dataset/data.yaml", epochs=100, imgsz=640, save_period=10)

Визуализация графиков обучения модели

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img_path = '/content/runs/detect/train/results.png'

img = mpimg.imread(img_path)
plt.imshow(img)
plt.axis('off')
plt.show()

# Конвертация под плату

Настраиваем окружение и скачиваем/устанавливаем все пакеты

In [ ]:
# Env setup
# Via google drive feature copy LicheeRV Nano dir to users drive;
!cp /content/drive/MyDrive/LicheeRV_Nano_Resources/tpu-mlir_v1.9.beta.0-110-gfdb73ce31-20240719.tar.gz tpu-mlir.tar.gz
!mkdir tpu_mlir
!tar -xzf tpu-mlir.tar.gz -C tpu_mlir --strip-components=1
!mv ./tpu_mlir/$(ls ./tpu_mlir) ./tpu_mlir/source
!wget https://github.com/sophgo/tpu-mlir/releases/download/v1.14/tpu_mlir-1.14-py3-none-any.whl
!sudo add-apt-repository ppa:deadsnakes/ppa -y
!sudo apt update
!sudo apt install python3.10
!wget https://bootstrap.pypa.io/get-pip.py
!python3.10 get-pip.py
!python3.10 -m pip install tpu_mlir-1.14-py3-none-any.whl
!python3.10 -m pip install flatbuffers
!python3.10 -m pip install ultralytics
!python3.10 -m pip install onnxsim
!python3.10 -m pip install matplotlib-inline
!python3.10 -m pip install onnxruntime
!wget https://raw.githubusercontent.com/ret7020/LicheeRVNano/refs/heads/master/Projects/Yolov8/export_to_onnx.py

In [ ]:
!cp /content/runs/detect/train/weights/best.pt /content/my_model.pt

Экспорт модели в ONNX

In [ ]:
!python3.10 export_to_onnx.py my_model.pt 640 640

Настройка конвертации модели под плату

In [ ]:
import os

# Set model input image size
os.environ['IMG_SIZE_W'] = "640"
os.environ['IMG_SIZE_H'] = "640"

# Path to dataset for calibration matrix
os.environ['CALIBRATION_DATASET_PATH'] = "/content/dataset/train/images"
os.environ['CALIBRATION_IMAGES_COUNT'] = "10"
os.environ['MODEL_NAME'] = "my_model"
os.environ['TEST_IMG'] = "/content/dataset/train/images/" + sorted(os.listdir('/content/dataset/train/images'))[0] # first file from dataset for some export tests

In [ ]:
!mkdir /content/work
%cd /content/work

In [ ]:
!export PYTHONPATH=/usr/bin/python3.10
!source ../tpu_mlir/source/envsetup.sh
!model_transform.py --model_name $MODEL_NAME --model_def "/content/${MODEL_NAME}.onnx" --input_shapes [[1,3,$IMG_SIZE_W,$IMG_SIZE_H]] --mean 0.0,0.0,0.0 --scale 0.0039216,0.0039216,0.0039216 --keep_aspect_ratio --pixel_format rgb --test_input $TEST_IMG --test_result "${MODEL_NAME}_top_outputs.npz" --mlir $MODEL_NAME.mlir

In [ ]:
!source ../tpu_mlir/source/envsetup.sh
!echo "${MODEL_NAME}_cali_table"
!run_calibration.py $MODEL_NAME.mlir \
--dataset $CALIBRATION_DATASET_PATH \
--input_num $CALIBRATION_IMAGES_COUNT \
-o "${MODEL_NAME}_cali_table"

In [ ]:
!mkdir result
!source ../tpu_mlir/source/envsetup.sh
!model_deploy.py \
--mlir $MODEL_NAME.mlir \
--quant_input \
--quant_output \
--quantize int8 \
--calibration_table "${MODEL_NAME}_cali_table" \
--processor cv181x \
--test_input "${MODEL_NAME}_in_f32.npz" \
--test_reference "${MODEL_NAME}_top_outputs.npz" \
--tolerance 0.85,0.45 \
--model "./result/${MODEL_NAME}.cvimodel"

Файл с итоговыми весами **/content/work/result/my_model.cvimodel**